In [6]:
# === Chunk 1: Setup & Daten laden (KORRIGIERTE VERSION) ===
# (Dieser Chunk lädt Prognosen UND Instrumente z_t)

import os, sys
from pathlib import Path
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

# --- 1. Pfad-Setup (Standard-Snippet) ---
def _locate_repo_root(start: Path) -> Path:
    cur = start.resolve()
    for _ in range(5):  # walk up to 5 levels
        if (cur / 'src').exists():
            return cur
        if cur.parent == cur: break
        cur = cur.parent
    return start.resolve()

NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = _locate_repo_root(NOTEBOOK_DIR)
os.environ['PROJECT_ROOT'] = str(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

try:
    from src.io_timesplits import load_target
    from src.config import STAGEB_DIR, PROCESSED
except ImportError:
    print("Konnte 'src' nicht importieren, füge es zum Pfad hinzu...")
    SRC_DIR = PROJECT_ROOT / "src"
    if str(SRC_DIR) not in sys.path:
        sys.path.append(str(SRC_DIR))
    from io_timesplits import load_target
    from config import STAGEB_DIR, PROCESSED


# --- 2. Lade Prognosen (Logik von Notebook 1) ---
# WICHTIG: BEIDE MODELLE MÜSSEN HIER AUFGELISTET SEIN!
MODEL_RUNS = {
    # "Key (wird zu Spaltenname)": "Ordnername in outputs/stageB/"
    "ET_All_Features": "extra_trees",
    "ET_Dynamic_FI": "extra_trees_dynamic_fi",
}

# (Funktion load_active_predictions von Notebook 1 kopieren)
def load_active_predictions(model_runs: dict) -> pd.DataFrame:
    """
    Lädt alle 'is_active=True' Prognosen und merged sie zu einem
    einzigen DataFrame mit y_true und den Prognosen.
    (FUNKTIONIERENDE VERSION)
    """
    all_model_dfs = []
    y_true_series = None
    for model_name_key, run_id in model_runs.items():
        # Pfad zur preds.csv (korrekt)
        pred_path = STAGEB_DIR / run_id / "monthly" / "preds.csv"
        if not pred_path.exists():
            print(f"WARNUNG: Datei nicht gefunden, überspringe: {pred_path}")
            continue
        # Lade die CSV, parse die KORREKTE Datumsspalte
        df = pd.read_csv(pred_path, parse_dates=["date_t_plus_1"])
        df_active = df[df["is_active"] == True].copy()
        if df_active.empty:
            print(f"WARNUNG: Keine 'is_active=True' Zeilen gefunden in: {pred_path}")
            continue
        df_active = df_active.set_index("date_t_plus_1")
        # Die Spalte heißt jetzt z.B. "pred_ET_All_Features"
        df_pred = df_active[["y_pred"]].rename(columns={"y_pred": f"pred_{model_name_key}"})
        all_model_dfs.append(df_pred)
        if y_true_series is None:
            y_true_series = df_active["y_true"].rename("y_true")
    if not all_model_dfs:
        raise FileNotFoundError("Keine Prognosedateien gefunden oder keine 'is_active=True' Zeilen.")
    df_merged = pd.concat(all_model_dfs, axis=1)
    if y_true_series is not None:
        df_merged = pd.concat([y_true_series, df_merged], axis=1)
    df_final = df_merged.dropna()
    if "y_true" in df_final.columns:
        cols = ["y_true"] + [col for col in df_final.columns if col != "y_true"]
        df_final = df_final[cols]
    return df_final

try:
    df_preds = load_active_predictions(MODEL_RUNS)
    print(f"\nDaten erfolgreich geladen. Shape: {df_preds.shape}")
except Exception as e:
    print(f"\nFEHLER beim Laden: {e}")


# --- 3. Instrumente (z_t) erstellen ---
# (Diese Funktion war bereits korrekt)
def create_instruments(y: pd.Series) -> pd.DataFrame:
    """Erstellt die F_t-1 messbaren Instrumente z_t."""
    y_lag1 = y.shift(1)
    q90_roll = y_lag1.abs().expanding(min_periods=60).quantile(0.9)
    z = pd.DataFrame(index=y.index)
    z['HighVol_t1'] = (y_lag1.abs() > q90_roll).astype(float)
    z['Neg_t1'] = (y_lag1 < 0).astype(float)
    # 'const' wird später von sm.add_constant hinzugefügt
    return z.dropna()

z_t = create_instruments(df_preds["y_true"])

# --- 4. Daten alignieren ---
# (Verluste und Instrumente auf gemeinsamen Index bringen)

# Verluste berechnen
y_true_np = df_preds["y_true"].to_numpy()
df_losses = pd.DataFrame(index=df_preds.index)
for col in df_preds.columns:
    if col.startswith("pred_"):
        model_name = col.replace("pred_", "")
        y_pred_np = df_preds[col].to_numpy()
        df_losses[model_name] = (y_true_np - y_pred_np) ** 2

# KORREKTUR: Beispielvergleich anpassen an die Keys aus MODEL_RUNS
model_A = "ET_All_Features"
model_B = "ET_Dynamic_FI"

# Verlustdifferenz d_t+1 = loss(A) - loss(B)
d_t1 = (df_losses[model_A] - df_losses[model_B]).rename("loss_diff")

# Alignment
data_cpa = pd.concat([d_t1, z_t], axis=1).dropna()
d_vec = data_cpa["loss_diff"]
z_mat = data_cpa[z_t.columns] # Nur Instrumente

print(f"CPA-Daten (aligniert): {data_cpa.shape}")
display(data_cpa.head())


Daten erfolgreich geladen. Shape: (166, 3)
CPA-Daten (aligniert): (166, 3)


,loss_diff,HighVol_t1,Neg_t1
date_t_plus_1,,,
2011-03-01,0.013922,0.0,0.0
2011-04-01,-0.002755,0.0,0.0
2011-05-01,-0.010989,0.0,0.0
2011-06-01,-1.271092,0.0,0.0
2011-07-01,-3.144864,0.0,1.0


In [4]:
# === Chunk 2: CPA Test (Regressions-basiert) (KORRIGIERT) ===

print(f"CPA Test: {model_A} (A) vs. {model_B} (B)")
print("H0: E[loss(A) - loss(B) | z_t] = 0  (Kein Modell ist konditional besser)")
print("H1: E[loss(A) - loss(B) | z_t] != 0 (Performance hängt von Regimen ab)\n")

# KORREKTUR 1: Spalten explizit trennen und Konstante vorne anfügen
instrument_cols = [col for col in z_mat.columns if col != 'const']
z_mat_reg = z_mat[instrument_cols]
z_mat_reg = sm.add_constant(z_mat_reg, prepend=True) # -> ['const', 'HighVol_t1', 'Neg_t1']

# KORREKTUR 2: HAC-Lags (q) konsistent zum Text (z.B. 3)
HAC_LAGS = 3

model_cpa = sm.OLS(d_vec, z_mat_reg).fit(cov_type='HAC', cov_kwds={'maxlags': HAC_LAGS})

print(model_cpa.summary())

# KORREKTUR 3: F-Test ist jetzt korrekt, da 'const' an Pos 0 ist
R_matrix = np.eye(len(model_cpa.params))[1:, :]

if R_matrix.shape[0] > 0: # Nur testen, wenn es nicht-konstante Regressoren gibt
    f_test = model_cpa.f_test(R_matrix)
    print(f"\nCPA F-Test (H0: Alle Koeffizienten außer 'const' sind 0):")

    # KORREKTUR HIER: .item() entfernt
    print(f"F-Statistik: {f_test.fvalue:.4f}")
    print(f"p-Wert: {f_test.pvalue:.4f}")

    if f_test.pvalue < 0.05:
        print("\n-> H0 verworfen. Die Modellperformance ist REGIM-ABHÄNGIG.")
    else:
        print("\n-> H0 nicht verworfen. Die Modellperformance ist REGIM-UNABHÄNGIG.")
else:
    print("\nKeine nicht-konstanten Instrumente für F-Test vorhanden.")

CPA Test: ET_All_Features (A) vs. ET_Dynamic_FI (B)
H0: E[loss(A) - loss(B) | z_t] = 0  (Kein Modell ist konditional besser)
H1: E[loss(A) - loss(B) | z_t] != 0 (Performance hängt von Regimen ab)

                            OLS Regression Results                            
Dep. Variable:              loss_diff   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     1.077
Date:                Sun, 09 Nov 2025   Prob (F-statistic):              0.343
Time:                        10:46:45   Log-Likelihood:                -601.17
No. Observations:                 166   AIC:                             1208.
Df Residuals:                     163   BIC:                             1218.
Df Model:                           2                                         
Covariance Type:                  HAC                                         
             